In [1]:
%pip install plotly scikit-learn numpy pandas nbformat pyhull

Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.datasets import make_blobs
import numpy as np
import json
from collections import defaultdict, namedtuple
import plotly.express as px
from pyhull.convex_hull import ConvexHull
from itertools import product

In [3]:
rs = np.random.RandomState(42)
num_cluster = 10
num_dim = 3

centres = [[rs.rand() for _ in range(num_dim)] for _ in range(num_cluster)]
sigmas = [rs.rand() for _ in range(10)]
blobs = make_blobs(
                   n_samples=100, 
                   n_features=num_dim,
                   centers=centres,
                   cluster_std=sigmas,
                   random_state=rs
                   ) 

In [4]:
point_clouds = [[] for _ in range(num_cluster)]
for sample, cluster in zip(blobs[0], blobs[1]):
    point_clouds[int(cluster)].append(sample.tolist())
inputs = [{"vertices" : points} for points in point_clouds]
with open("synthetic.json", "wt") as out_file:
    json.dump(inputs, out_file, indent=2)

In [5]:
with open("synthetic_out.json", "rt") as in_file:
    polys = json.load(in_file)

In [6]:
print(len(polys[5]["essential_indices"]))

8


In [7]:
poly_ind = 0
vertices = np.array(polys[poly_ind]["vertices"])
constant = np.ones([len(vertices), 1], dtype=float)
vertices = np.concatenate([constant, vertices], axis=1)
certs = {extreme: np.array(cert) for extreme, cert in polys[poly_ind]["essential_certs"].items()}

for vertex, cert in certs.items():
    scores = vertices @ cert
    print(scores, vertex, np.argmin(scores))

[ 1.00000000e+00  7.56728014e-13 -1.44658287e+00 -6.76379086e-01
 -1.10230577e+00 -6.54650143e-01 -1.42974848e+00 -5.18449522e-01
 -6.90129634e-01 -4.68236561e-14] 0 2
[-2.08959101e+00 -1.10755849e-11 -1.63927999e+00 -2.65557562e+01
 -1.82271975e-11 -7.55564118e+00 -3.76643191e+01  1.00000000e+00
 -1.26316542e+01  3.94528854e-12] 7 6
[-2.30012395e+00 -1.78923630e+00  1.00000000e+00 -4.04672882e-02
 -2.20712337e-13 -4.56830358e-01  1.45528034e-12 -5.89222875e-01
 -4.52748949e-13 -5.48354881e-01] 2 0
[-8.73745520e-13 -2.05280426e+00  1.92734717e-13 -1.18169669e-01
 -1.30985690e+00 -9.82697407e-01 -2.57055466e+00 -8.16832280e-01
 -4.60173566e-13  1.00000000e+00] 9 6
[-3.97597632e+00 -2.69828604e-12 -4.56079619e-13 -4.15543815e+00
  1.00000000e+00 -9.77760803e-01 -1.77429335e+00 -3.22297087e-01
 -2.55480151e+00 -3.14997073e+00] 4 3
[-7.29431196e-01  1.04172226e-12 -2.75665424e-01  2.22044605e-15
  7.64721619e-13 -5.81603188e-02  1.00000000e+00 -2.63284624e-01
 -2.47089302e-01 -8.53040640e-

In [8]:
poly_ind = 6
poly = polys[poly_ind]

dims = ["x", "y", "z"]
vertices_by_col = {dims[i]: list(col) for i, col in enumerate(zip(*poly["vertices"]))}
vertices_by_col["extreme"] = [str(i in poly["essential_indices"]) for i in range(len(poly["vertices"]))]
vertices_by_col["text"] = [str(i) for i in range(len(vertices_by_col["x"]))]

# Plot the edges

lines = {}
for start_vertex, adjacent in poly["adjacency"].items():
    for end_vertex in adjacent:
        line = frozenset((int(start_vertex), int(end_vertex)))
        count = lines.get(line, 0)
        lines[line] = count + 1
        
for line, count in lines.items():
    #print(line, count)
    assert count == 2, "Should be dupes"

lines_by_col = {dim: [] for dim in dims}
lines_by_col["line"] = []
for line in lines:
    for vertex in line:
        for dim in dims:
            lines_by_col[dim].append(vertices_by_col[dim][vertex])
        lines_by_col["line"].append(str(set(line)))

#(df=vertices_by_col, x="x", y="y", z="z", color="extreme", text="text")
fig = px.line_3d(lines_by_col, x="x", y="y", z="z", color="line")
#fig.add_scatter3d(**{dim: vertices_by_col[dim] for dim in dims})
fig.show()

In [23]:
with open("states_out.jsonl") as states_file:
    states_data = [json.loads(line) for line in states_file]

RSOut = namedtuple("RSOut", ["param", "decomp"])

states  = [RSOut(np.array(item["param"]["data"]), np.array(item["minkowski_decomp"])) for item in states_data]

for state in states:
    print(state)


RSOut(param=array([0.47774499, 0.67478222, 0.56251994]), decomp=array([6, 9, 5, 5, 7, 1, 3, 7, 6, 4]))
RSOut(param=array([0.70896346, 0.58056507, 0.40039358]), decomp=array([6, 9, 6, 1, 7, 1, 3, 7, 6, 4]))
RSOut(param=array([0.81030542, 0.4543805 , 0.37005877]), decomp=array([6, 9, 6, 1, 7, 1, 7, 7, 6, 4]))
RSOut(param=array([0.88720398, 0.26553725, 0.37730501]), decomp=array([6, 9, 6, 1, 7, 1, 7, 7, 2, 4]))
RSOut(param=array([0.91989796, 0.33930686, 0.19661788]), decomp=array([6, 9, 6, 1, 7, 1, 7, 7, 2, 2]))
RSOut(param=array([0.95057846, 0.19478552, 0.24178337]), decomp=array([6, 9, 6, 1, 7, 1, 7, 7, 2, 6]))
RSOut(param=array([0.9180898 , 0.39338812, 0.04854807]), decomp=array([1, 9, 6, 1, 7, 1, 7, 7, 2, 6]))
RSOut(param=array([0.97158834, 0.18301201, 0.15007566]), decomp=array([6, 9, 6, 1, 7, 1, 7, 3, 2, 6]))
RSOut(param=array([ 0.98408051,  0.1232688 , -0.1280248 ]), decomp=array([1, 9, 6, 1, 7, 1, 7, 3, 2, 6]))
RSOut(param=array([ 0.94762111, -0.10816601, -0.30052347]), decomp=arr

In [24]:
np_poly = np.array(point_clouds)

decomps_counter = 0
decomps = {}

for state in states:
    param = np.array(state.param)
    scores = np_poly @ param
    maximised = np.argmax(scores, axis=1)
    #print("python: ", maximised)
    #print("rs out: ", state.decomp)
    assert(np.array_equal(state.decomp, maximised))
    
    decomp_key = tuple(state.decomp)
    if decomp_key not in decomps:
        decomps[decomp_key] = decomps_counter
        decomps_counter += 1
    print(f"{decomps[decomp_key]}, ", end="")

#print(f"Num decomps {len(decomps)}, num states {len(states)}")

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 

In [20]:
params = np.stack([s.param for s in states])

fig = px.scatter_3d(x=params[:,0], y=params[:,1], z=params[:,2])
fig.show()

Now we try to compute a Minkowski sum directly through qhull

- I'm giving up on using qhull as its numerical stability mechanisms mean that we get different results

In [78]:
def qhull(cloud):
    hull = ConvexHull(cloud)
    return sorted({v for h_v in ConvexHull(cloud).vertices for v in h_v})


hulls = [qhull(cloud) for cloud in point_clouds]

MinkSumPoint = namedtuple("MinkSumPoint", ["point", "decomp"])


def mink_sum(point_clouds, hulls):
    hull_points = [
        [MinkSumPoint(cloud[vertex], [vertex]) for vertex in hull]
        for cloud, hull in zip(point_clouds, hulls)
    ]
    hull_points.reverse()
    mink_sum = hull_points.pop()
    for operand in hull_points:
        print(f"Input lens {len(mink_sum)}, {len(operand)}")
        point_product = [
            MinkSumPoint([x_a + x_b for x_a, x_b in zip(a.point, b.point)], a.decomp + b.decomp)
            for a, b in product(mink_sum, operand)
        ]
        mink_sum = [point_product[i] for i in qhull([point for point, _ in point_product])]
        print(f"Lengths {len(point_product)}, {len(mink_sum)}")
    return mink_sum

num_polys = 10
mink_sum = mink_sum(point_clouds[:num_polys], hulls[:num_polys])
print(len(mink_sum))

for vertex in mink_sum:
    print(vertex.decomp)

Input lens 8, 9
Lengths 72, 29
Input lens 29, 7
Lengths 203, 58
Input lens 58, 7
Lengths 406, 93
Input lens 93, 8
Lengths 744, 158
Input lens 158, 8
Lengths 1264, 234
Input lens 234, 10
Lengths 2340, 332
Input lens 332, 8
Lengths 2656, 435
Input lens 435, 8
Lengths 3480, 549
Input lens 549, 9
Lengths 4941, 696
696
[0, 0, 7, 1, 1, 3, 1, 8, 9, 1]
[0, 0, 7, 9, 1, 3, 1, 8, 9, 1]
[0, 0, 9, 1, 0, 3, 1, 8, 9, 1]
[0, 0, 9, 1, 1, 3, 1, 8, 9, 1]
[0, 0, 9, 1, 1, 3, 1, 9, 9, 1]
[0, 0, 9, 1, 1, 4, 1, 8, 9, 1]
[0, 0, 9, 1, 1, 4, 1, 9, 9, 1]
[0, 0, 9, 1, 1, 4, 1, 9, 9, 3]
[0, 0, 9, 1, 1, 4, 2, 9, 9, 1]
[0, 0, 9, 1, 1, 4, 2, 9, 9, 3]
[0, 0, 9, 9, 0, 3, 1, 8, 9, 1]
[0, 0, 9, 9, 1, 3, 1, 8, 9, 1]
[0, 3, 7, 1, 1, 2, 1, 6, 0, 3]
[0, 3, 7, 1, 1, 2, 1, 6, 2, 3]
[0, 3, 7, 1, 1, 2, 3, 6, 2, 3]
[0, 3, 7, 1, 1, 2, 5, 6, 2, 3]
[0, 3, 7, 1, 1, 2, 8, 6, 2, 3]
[0, 3, 7, 1, 1, 2, 8, 8, 2, 3]
[0, 3, 7, 1, 1, 3, 1, 6, 0, 3]
[0, 3, 7, 1, 1, 3, 1, 6, 2, 3]
[0, 3, 7, 1, 1, 3, 1, 8, 0, 1]
[0, 3, 7, 1, 1, 3, 1, 8, 0, 3]
[0